In [1]:
from datasets import load_dataset
import nltk
from tqdm import tqdm
import re
import spacy
from spacy.tokenizer import _get_regex_pattern

c:\Users\Maxim Zubarev\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\cuda\__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


Импортирование датасета INCPEC с hugging face

In [2]:
from datasets import load_dataset
import datasets
#!!!this dataset can be imported only with datasets library version < 4.0.0!!!
print (datasets.__version__)

# load the inspec dataset
dataset = load_dataset('taln-ls2n/inspec')

2.16.0


Using the latest cached version of the module from C:\Users\Maxim Zubarev\.cache\huggingface\modules\datasets_modules\datasets\taln-ls2n--inspec\0ae146cabe770846946b3279b4c751efe0aca2dd68b3f24427d4624cd22bb20d (last modified on Thu Dec 11 16:48:11 2025) since it couldn't be found locally at taln-ls2n/inspec, or remotely on the Hugging Face Hub.


Соединение частей датасетов в один

In [3]:
train_dataset = dataset['train']
test_dataset = dataset['test']
val_dataset = dataset['validation']

In [4]:
full_dataset = datasets.concatenate_datasets([train_dataset, test_dataset, val_dataset])

Импортирование и редактирование языковой модели под слова с дефисами

In [5]:
nlp = spacy.load("en_core_web_sm")
re_token_match = _get_regex_pattern(nlp.Defaults.token_match)
re_token_match = f"({re_token_match}|\w+-\w+)"
nlp.tokenizer.token_match = re.compile(re_token_match).match

<>:3: SyntaxWarning: invalid escape sequence '\w'
<>:3: SyntaxWarning: invalid escape sequence '\w'
C:\Users\Maxim Zubarev\AppData\Local\Temp\ipykernel_3992\2674231081.py:3: SyntaxWarning: invalid escape sequence '\w'
  re_token_match = f"({re_token_match}|\w+-\w+)"


Обработка данных: приведение заголовка и введения в один текст, стеммизация ключевых слов

In [6]:
def preprocessing(dataset, nlp):
    keyphrases = []
    text = []
    for sample in tqdm(dataset):
        sample_keyphrases = []
        text.append(sample["title"] + ". " + sample["abstract"])
        for keyphrase in sample["keyphrases"]:
            tokens = [token.text for token in nlp(keyphrase)]
            stemmed = [nltk.stem.snowball.SnowballStemmer('porter').stem(token.lower()) for token in tokens]
            sample_keyphrases.append(" ".join(stemmed))
        keyphrases.append(sample_keyphrases)
    return text, keyphrases

In [7]:
text, references = preprocessing(full_dataset, nlp)

100%|██████████| 2000/2000 [01:53<00:00, 17.58it/s]


Извлечение слов. Максимальная длина n-грама 3. KeyBERT использует пользовательский векторизатор, учитывающий грамматическую связь слов

In [ ]:
def extraction(model, text, references, stemmer, vectorizer=None):
    output = []
    for doc, reference in tqdm(zip(text, references)):
        n = len(reference)
        match model.__class__.__name__:
            case 'KeyBERT':
                keywords = model.extract_keywords(
                    doc,
                    keyphrase_ngram_range=(1, 3),
                    vectorizer=vectorizer,
                    use_mmr=True,
                    diversity=0.7,
                    top_n=n * 2)
                stemmed = []
                for kw, score in keywords:
                    words = kw.split()
                    stemmed_words = [stemmer.stem(word.lower()) for word in words]
                    stemmed_phrase = " ".join(stemmed_words)    
                    stemmed.append(stemmed_phrase)
                output.append(stemmed[:n])
            case 'Rake':
                model.extract_keywords_from_text(doc)
                keywords = model.get_ranked_phrases()
                stemmed = []
                for kw in keywords:
                    words = kw.split()
                    stemmed_words = [stemmer.stem(word.lower()) for word in words]
                    stemmed_phrase = " ".join(stemmed_words)    
                    stemmed.append(stemmed_phrase)
                output.append(stemmed[:n])
            case _:
                model.load_document(input=doc)
                model.grammar_selection(grammar="NP: {<ADJ>*<NOUN|PROPN>+}")
                model.candidate_weighting()
                output.append([keyword for keyword, rank in model.get_n_best(n=n, stemming=True)])
    return output

In [10]:
from pke.unsupervised import TextRank, TopicRank, YAKE
from CustomVectorizer import CustomVectorizer
from nltk.corpus import stopwords
from keybert import KeyBERT
from rake_nltk import Rake
nltk.download('stopwords')
nltk.download('punkt')
stemmer = stemmer = nltk.stem.snowball.SnowballStemmer('porter')
nlp = spacy.load("en_core_web_sm")
vectorizer = CustomVectorizer(
    nlp_model=nlp,
    ngram_range=(1, 3),
    stop_words='english')
outputs = {}
for model in [KeyBERT, Rake, TextRank, TopicRank, YAKE]:
    extractor = model()
    if extractor.__class__.__name__ == 'Rake':
        extractor = model(stopwords=stopwords.words('english'), language='english', max_length=3)
    outputs[extractor.__class__.__name__] = extraction(extractor, text, references, stemmer, vectorizer=vectorizer)
outputs

[nltk_data] Downloading package stopwords to C:\Users\Maxim
[nltk_data]     Zubarev\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Maxim
[nltk_data]     Zubarev\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2000it [04:16,  7.78it/s]
2000it [00:02, 686.58it/s]
2000it [24:26,  1.36it/s]
2000it [24:08,  1.38it/s]
2000it [24:29,  1.36it/s]


{'KeyBERT': [['lattic ga algorithm',
   'nmr techniqu',
   'qubit quantum inform',
   'puls techniqu',
   'densiti',
   'processor',
   'ideal simul',
   'reson'],
  ['sma fund', 'small bank', 'outsourc', 'account', 'servic', 'invest'],
  ['diagnost system',
   'multipl simultan failur',
   'complex industri applic',
   'design cycl',
   'modular'],
  ['particular minimum cliqu',
   'graph invari',
   'induc odd cycl',
   'stabl set',
   'special structur',
   'cardin'],
  ['packetvideo',
   'stream wireless market',
   'content provid',
   'semiconductor vendor',
   'pictur expert group',
   'encod',
   'technolog',
   'handheld wireless termin'],
  ['arbitrari quantum dynam',
   'univers quantum comput',
   'composit quantum system',
   'univers simul',
   'dimension state space',
   'hamiltonian',
   'qudit',
   'compon system',
   'local unitari',
   'd dimens',
   'interact',
   'state'],
  ['dynam vibrat absorb',
   'optim criterion',
   'element type dva',
   'power spectrum',
 

In [ ]:
import pickle

with open('outputs_INSPEC.pkl', 'wb') as f:
    pickle.dump(outputs, f)

In [ ]:
import pickle

with open('outputs_INSPEC.pkl', 'rb') as f:
    outputs = pickle.load(f)

Оценка моделей F1-метрикой

In [11]:
def evaluation(predictions, references):
    precision = len(set(predictions) & set(references)) / len(set(predictions))
    recall = len(set(predictions) & set(references)) / len(set(references))
    F1_metrics = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0
    return F1_metrics


In [12]:
import numpy as np
evals = {}
for model in outputs:
    scores = []
    for output, reference in zip(outputs[model], references):
        scores.append(evaluation(output, reference))
    avg_score = np.mean(scores, axis = 0)
    evals[model] = round(avg_score, 2)
evals

{'KeyBERT': np.float64(0.26),
 'Rake': np.float64(0.22),
 'TextRank': np.float64(0.34),
 'TopicRank': np.float64(0.29),
 'YAKE': np.float64(0.34)}